In [37]:
import pbpstats
import pandas as pd
from pbpstats.client import Client
from nba_api.stats.endpoints import playercareerstats, commonplayerinfo, playerprofilev2
from nba_api.stats.static import players
from nba_api.stats.static import teams
import time
from alive_progress import alive_bar

In [36]:
#!pip install pbpstats --upgrade
#!pip install nba_api
#!pip install tqdm
#!pip install jupyter --upgrade
#!pip install ipywidgets --user
#!pip install alive-progress

  Created wheel for grapheme: filename=grapheme-0.6.0-py3-none-any.whl size=210096 sha256=64556f60620370e9d70baa57f30a319d46e6109cba3012cd1c54705deeb00596
  Stored in directory: c:\users\gabri\appdata\local\pip\cache\wheels\2d\08\6b\126ea9009f7482fd53a78d0db2ece5aca70af8f4a30445386b
Successfully built grapheme


In [3]:
# get_players returns a list of dictionaries, each representing a player.
nba_players = players.get_players()
print('Number of players fetched: {}'.format(len(nba_players)))
nba_players[:2]

Number of players fetched: 4723


[{'id': 76001,
  'full_name': 'Alaa Abdelnaby',
  'first_name': 'Alaa',
  'last_name': 'Abdelnaby',
  'is_active': False},
 {'id': 76002,
  'full_name': 'Zaid Abdul-Aziz',
  'first_name': 'Zaid',
  'last_name': 'Abdul-Aziz',
  'is_active': False}]

In [4]:
# Anthony Davis
career = playercareerstats.PlayerCareerStats(player_id='203076')
career.get_data_frames()[0]

,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
0,203076,2012-13,00,1610612740,NOH,20.0,64,60,1846.0,349,...,0.751,165,357,522,63,75,112,89,158,867
1,203076,2013-14,00,1610612740,NOP,21.0,67,66,2358.0,522,...,0.791,207,466,673,105,89,189,109,200,1394
2,203076,2014-15,00,1610612740,NOP,22.0,68,68,2455.0,642,...,0.805,173,523,696,149,100,200,95,141,1656
3,203076,2015-16,00,1610612740,NOP,23.0,61,61,2164.0,560,...,0.758,130,497,627,116,78,125,121,148,1481
4,203076,2016-17,00,1610612740,NOP,24.0,75,75,2708.0,770,...,0.802,172,712,884,157,94,167,181,168,2099
5,203076,2017-18,00,1610612740,NOP,25.0,75,75,2727.0,780,...,0.828,187,644,831,174,115,193,162,159,2110
6,203076,2018-19,00,1610612740,NOP,26.0,56,56,1850.0,530,...,0.794,174,498,672,218,88,135,112,132,1452
7,203076,2019-20,00,1610612747,LAL,27.0,62,62,2131.0,551,...,0.846,142,435,577,200,91,143,154,156,1618
8,203076,2020-21,00,1610612747,LAL,28.0,36,36,1162.0,301,...,0.738,62,224,286,110,45,59,74,60,786
9,203076,2021-22,00,1610612747,LAL,29.0,40,40,1404.0,370,...,0.713,106,288,394,122,49,90,82,97,927


In [5]:
ts = teams.get_teams()

In [6]:
# get players that played in NBA season 2021-22
# calculate career averages, group by team
# take care of players that were traded. Remove entry for team for which they played the least.
# export JSON file, see template

In [7]:
nba_players = [i for i in players.get_players() if i['is_active']==True]

In [49]:
# get stats
def get_last_season_players(player_ids):
    
    custom_headers = {
    'Host': 'stats.nba.com',
    'Connection': 'keep-alive',
    'Cache-Control': 'max-age=0',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.86 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9',
    }
    
    l = []
    with alive_bar(total=len(player_ids), force_tty=True) as bar:
        for id2 in player_ids:
            bar()
            #career = playercareerstats.PlayerCareerStats(player_id=id2)
            career = playerprofilev2.PlayerProfileV2(player_id=str(id2))
            df = career.get_data_frames()[0]
            if '2021-22' in df['SEASON_ID'].unique():
                l.append(df)
    return l

In [50]:
player_ids = [i['id'] for i in nba_players]
last_season_players = get_last_season_players(player_ids)

|████████████████████████████████████████| 587/587 [100%] in 17:18.1 (0.57/s)                                           


In [53]:
last_season_players[-1]

,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
0,1627826,2016-17,00,1610612747,LAL,20.0,38,11,609,126,...,0.653,41,118,159,30,14,33,30,66,284
1,1627826,2017-18,00,1610612747,LAL,21.0,43,0,410,61,...,0.765,45,78,123,25,8,15,26,47,161
2,1627826,2018-19,00,1610612747,LAL,22.0,33,12,516,112,...,0.864,54,108,162,25,4,27,33,73,281
3,1627826,2018-19,00,1610612746,LAC,22.0,26,25,524,100,...,0.733,61,139,200,38,10,24,37,64,244
4,1627826,2018-19,00,0,TOT,22.0,59,37,1039,212,...,0.802,115,247,362,63,14,51,70,137,525
5,1627826,2019-20,00,1610612746,LAC,23.0,72,70,1326,236,...,0.747,197,346,543,82,16,66,61,168,596
6,1627826,2020-21,00,1610612746,LAC,24.0,72,33,1609,257,...,0.789,189,330,519,90,24,62,81,187,650
7,1627826,2021-22,00,1610612746,LAC,25.0,76,76,1852,310,...,0.727,217,427,644,120,36,77,114,203,785


In [40]:
with alive_bar(total=5, force_tty=True) as bar:
    for i in range(5):
        bar()
        time.sleep(1)

|████████████████████████████████████████| 5/5 [100%] in 5.2s (0.96/s)                                                  


In [55]:
last_season_players[3]

,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
0,1630583,2021-22,00,1610612763,MEM,21.0,32,0,360,53,...,0.625,33,54,87,21,6,10,16,36,132


In [59]:
df = pd.concat([i for i in last_season_players])

In [61]:
df.to_csv('all_players.csv', index=False)

In [1]:
nba_players[0]

NameError: name 'nba_players' is not defined